#### Imports

In [1]:
import math
import os
import random
from typing import Any, Dict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from ipywidgets import IntSlider, interact
from sklearn.metrics import classification_report, confusion_matrix
from torch.nn import Parameter
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms
from torchmetrics.image import StructuralSimilarityIndexMeasure, PeakSignalNoiseRatio
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import wandb

wandb.login()

from pytorch_lightning import seed_everything
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import RichProgressBar, ModelCheckpoint
from pytorch_lightning.tuner import Tuner

import kornia

cuda


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: nbennewiz to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
def set_seed(seed=100):
    seed_everything(seed, workers=True)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For all GPUs
    np.random.seed(seed)
    random.seed(seed)

set_seed(100)

Seed set to 100


In [3]:
# Small function for normalizing the already standardized data
"""from rich.progress import track
load_root="../QualityFiltered"
save_root="../NormalizedQualityFiltered"
for volume_name in track([path for path in os.listdir(root)]):
    #load
    volume_seq = np.load(os.path.join(load_root, volume_name))
    #min-max scaling
    volume_seq = (volume_seq - volume_seq.min())/(volume_seq.max() - volume_seq.min())
    #save
    np.save(os.path.join(save_root, volume_name), volume_seq)"""

'from rich.progress import track\nload_root="../QualityFiltered"\nsave_root="../NormalizedQualityFiltered"\nfor volume_name in track([path for path in os.listdir(root)]):\n    #load\n    volume_seq = np.load(os.path.join(load_root, volume_name))\n    #min-max scaling\n    volume_seq = (volume_seq - volume_seq.min())/(volume_seq.max() - volume_seq.min())\n    #save\n    np.save(os.path.join(save_root, volume_name), volume_seq)'

#### Dataset

In [4]:
class Dataset2D(Dataset):
    def __init__(
        self, data_paths, context_window=4, prediction_window=1, transform=None
    ):
        self.data_paths = data_paths
        self.context_window = context_window
        self.prediction_window = prediction_window
        self.transform = transform
        self.depth = None
        # For every path to a volume sequence in .npy
        self.data_attributes = []
        test = np.load(data_paths[0])
        for data_path in self.data_paths:
            for h in range(test.shape[1]):
                for t in range(len(test)-self.context_window - self.prediction_window+1):
                    # file_path, t, h
                    self.data_attributes.append([data_path, t, h])

    def __len__(self):
        return len(self.data_attributes)

    def __getitem__(self, idx):
        t = self.data_attributes[idx][1]
        h = self.data_attributes[idx][2]
        volume_seq = torch.from_numpy(np.load(self.data_attributes[idx][0]))
        return (
            volume_seq[t:t+self.context_window, h].unsqueeze(1),
            volume_seq[t+self.context_window:t+self.context_window+self.prediction_window, h].unsqueeze(1)
        )

class Dataset3D(Dataset):
    def __init__(self, data_paths, context_window=4, prediction_window=1, transform=None):
        self.data_paths = data_paths
        self.context_window = context_window
        self.prediction_window = prediction_window
        self.transform = transform

        self.data_attributes = []
        test = np.load(self.data_paths[0])
        for data_path in self.data_paths:
            for t in range(test.shape[0]-self.context_window-self.prediction_window+1):
                # file_path, t
                self.data_attributes.append([data_path, t])
        
    def __len__(self):
        return len(self.data_attributes)
    
    def __getitem__(self, idx):
        t = self.data_attributes[idx][1]
        volume_seq = torch.from_numpy(np.load(self.data_attributes[idx][0]))
        return (
            volume_seq[t:t+self.context_window].unsqueeze(1), 
            volume_seq[t+self.context_window:t+self.context_window+self.prediction_window].unsqueeze(1)
        )

In [5]:
def seed_worker(worker_id):
    """Ensures worker processes get the same seed"""
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

class VolumeDataModule(pl.LightningDataModule):
    def __init__(self, root, batch_size=4, sequence_length=4, prediction_length=1, num_workers=0, drop_last=False, pin_memory=False, train_split=0.8, val_split=0.1, test_split=0.1):
        super().__init__()
        self.root = root
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.prediction_length = prediction_length
        self.num_workers = num_workers
        self.drop_last = drop_last
        self.pin_memory = pin_memory
        
        self.train_split = train_split
        self.val_split = val_split
        self.test_split = test_split
        
        self.train_paths = None
        self.val_paths = None
        self.test_paths = None

    def setup(self, stage=None):
        data_paths = [os.path.join(self.root, path) for path in os.listdir(self.root)]
        total_size = len(data_paths)
    
        # Normalize splits if they don’t sum to 1
        split_sum = self.train_split + self.val_split + self.test_split
        if split_sum != 1.0:
            self.train_split /= split_sum
            self.val_split /= split_sum
            self.test_split /= split_sum
            print(f"Normalized splits to: train={self.train_split:.2f}, val={self.val_split:.2f}, test={self.test_split:.2f}")
    
        # Compute dataset sizes
        train_size = int(total_size * self.train_split)
        val_size = int(total_size * self.val_split)
        test_size = total_size - train_size - val_size  # Ensure all data is used
    
        # Error handling: Ensure valid split sizes
        if train_size <= 0 or val_size <= 0 or test_size <= 0:
            raise ValueError(f"Invalid dataset splits: train={train_size}, val={val_size}, test={test_size}. Check your split values.")
    
        # Perform random split
        self.train_paths, self.val_paths, self.test_paths = random_split(data_paths, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42))

        self.train_dataset = Dataset2D(self.train_paths, self.sequence_length, self.prediction_length)
        self.val_dataset = Dataset2D(self.val_paths, self.sequence_length, self.prediction_length)
        self.test_dataset = Dataset2D(self.test_paths, self.sequence_length, self.prediction_length)
    
        self.val_dataset_3d = Dataset3D(self.val_paths, self.sequence_length, self.prediction_length)
        self.test_dataset_3d = Dataset3D(self.test_paths, self.sequence_length, self.prediction_length)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers, drop_last=self.drop_last, pin_memory=self.pin_memory, worker_init_fn=seed_worker)

    def val_dataloader(self):
        return [
            DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers, drop_last=self.drop_last, pin_memory=self.pin_memory, worker_init_fn=seed_worker),
            DataLoader(self.val_dataset_3d, batch_size=self.batch_size, num_workers=self.num_workers, drop_last=self.drop_last, pin_memory=self.pin_memory, worker_init_fn=seed_worker),
        ]

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=self.num_workers, drop_last=self.drop_last, pin_memory=self.pin_memory, worker_init_fn=seed_worker)

    def val_dataloader_3d(self):
        return DataLoader(self.val_dataset_3d, batch_size=self.batch_size, num_workers=self.num_workers, drop_last=self.drop_last, pin_memory=self.pin_memory, worker_init_fn=seed_worker)
    
    def test_dataloader_3d(self):
        return DataLoader(self.test_dataset_3d, batch_size=self.batch_size, num_workers=self.num_workers, drop_last=self.drop_last, pin_memory=self.pin_memory, worker_init_fn=seed_worker)

    def teardown(self, stage=None):
        if stage == "fit" or stage is None:
            pass
            #print("Cleaning up after training...")

        if stage == "test" or stage is None:
            pass
            #print("Cleaning up after testing...")

        if stage == "validate" or stage is None:
            pass
            #print("Cleaning up after validation...")

        # Free memory by deleting large datasets
        del self.train_dataset
        del self.val_dataset
        del self.test_dataset
        del self.val_dataset_3d
        del self.test_dataset_3d

#### Model

In [6]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, transpose=False, act_norm=False):
        super(BasicConv2d, self).__init__()
        self.act_norm=act_norm
        if not transpose:
            self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        else:
            self.conv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,output_padding=stride //2 )
        self.norm = nn.GroupNorm(2, out_channels)
        self.act = nn.LeakyReLU(0.2, inplace=True)

    def forward(self, x):
        y = self.conv(x)
        if self.act_norm:
            y = self.act(self.norm(y))
        return y


class ConvSC(nn.Module):
    def __init__(self, C_in, C_out, stride, transpose=False, act_norm=True):
        super(ConvSC, self).__init__()
        if stride == 1:
            transpose = False
        self.conv = BasicConv2d(C_in, C_out, kernel_size=3, stride=stride,
                                padding=1, transpose=transpose, act_norm=act_norm)

    def forward(self, x):
        y = self.conv(x)
        return y


class GroupConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, groups, act_norm=False):
        super(GroupConv2d, self).__init__()
        self.act_norm = act_norm
        if in_channels % groups != 0:
            groups = 1
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,groups=groups)
        self.norm = nn.GroupNorm(groups,out_channels)
        self.activate = nn.LeakyReLU(0.2, inplace=True)
    
    def forward(self, x):
        y = self.conv(x)
        if self.act_norm:
            y = self.activate(self.norm(y))
        return y


class Inception(nn.Module):
    def __init__(self, C_in, C_hid, C_out, incep_ker=[3,5,7,11], groups=8):        
        super(Inception, self).__init__()
        self.conv1 = nn.Conv2d(C_in, C_hid, kernel_size=1, stride=1, padding=0)
        layers = []
        for ker in incep_ker:
            layers.append(GroupConv2d(C_hid, C_out, kernel_size=ker, stride=1, padding=ker//2, groups=groups, act_norm=True))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        y = 0
        for layer in self.layers:
            y += layer(x)
        return y

def stride_generator(N, reverse=False):
    strides = [1, 2]*10
    if reverse: return list(reversed(strides[:N]))
    else: return strides[:N]

class Encoder(nn.Module):
    def __init__(self,C_in, C_hid, N_S):
        super(Encoder,self).__init__()
        strides = stride_generator(N_S)
        self.enc = nn.Sequential(
            ConvSC(C_in, C_hid, stride=strides[0]),
            *[ConvSC(C_hid, C_hid, stride=s) for s in strides[1:]]
        )
    
    def forward(self,x):# B*4, 3, 128, 128
        enc1 = self.enc[0](x)
        latent = enc1
        for i in range(1,len(self.enc)):
            latent = self.enc[i](latent)
        return latent,enc1


class Decoder(nn.Module):
    def __init__(self,C_hid, C_out, N_S):
        super(Decoder,self).__init__()
        strides = stride_generator(N_S, reverse=True)
        self.dec = nn.Sequential(
            *[ConvSC(C_hid, C_hid, stride=s, transpose=True) for s in strides[:-1]],
            ConvSC(2*C_hid, C_hid, stride=strides[-1], transpose=True)
        )
        self.readout = nn.Conv2d(C_hid, C_out, 1)
    
    def forward(self, hid, enc1=None):
        for i in range(0,len(self.dec)-1):
            hid = self.dec[i](hid)
        Y = self.dec[-1](torch.cat([hid, enc1], dim=1))
        Y = self.readout(Y)
        return Y

class Mid_Xnet(nn.Module):
    def __init__(self, channel_in, channel_hid, N_T, incep_ker = [3,5,7,11], groups=8):
        super(Mid_Xnet, self).__init__()

        self.N_T = N_T
        enc_layers = [Inception(channel_in, channel_hid//2, channel_hid, incep_ker= incep_ker, groups=groups)]
        for i in range(1, N_T-1):
            enc_layers.append(Inception(channel_hid, channel_hid//2, channel_hid, incep_ker= incep_ker, groups=groups))
        enc_layers.append(Inception(channel_hid, channel_hid//2, channel_hid, incep_ker= incep_ker, groups=groups))

        dec_layers = [Inception(channel_hid, channel_hid//2, channel_hid, incep_ker= incep_ker, groups=groups)]
        for i in range(1, N_T-1):
            dec_layers.append(Inception(2*channel_hid, channel_hid//2, channel_hid, incep_ker= incep_ker, groups=groups))
        dec_layers.append(Inception(2*channel_hid, channel_hid//2, channel_in, incep_ker= incep_ker, groups=groups))

        self.enc = nn.Sequential(*enc_layers)
        self.dec = nn.Sequential(*dec_layers)

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.reshape(B, T*C, H, W)

        # encoder
        skips = []
        z = x
        for i in range(self.N_T):
            z = self.enc[i](z)
            if i < self.N_T - 1:
                skips.append(z)

        # decoder
        z = self.dec[0](z)
        for i in range(1, self.N_T):
            z = self.dec[i](torch.cat([z, skips[-i]], dim=1))

        y = z.reshape(B, T, C, H, W)
        return y


class SimVP(nn.Module):
    def __init__(self, shape_in, hid_S=16, hid_T=256, N_S=4, N_T=8, incep_ker=[3,5,7,11], groups=8):
        super(SimVP, self).__init__()
        T, C, H, W = shape_in
        self.enc = Encoder(C, hid_S, N_S)
        self.hid = Mid_Xnet(T*hid_S, hid_T, N_T, incep_ker, groups)
        self.dec = Decoder(hid_S, C, N_S)


    def forward(self, x_raw):
        B, T, C, H, W = x_raw.shape
        x = x_raw.view(B*T, C, H, W)

        embed, skip = self.enc(x)
        _, C_, H_, W_ = embed.shape

        z = embed.view(B, T, C_, H_, W_)
        hid = self.hid(z)
        hid = hid.reshape(B*T, C_, H_, W_)

        Y = self.dec(hid, skip)
        Y = Y.reshape(B, T, C, H, W)
        return Y

#### Pl Model

In [7]:
class HuberSSIMLoss2D(nn.Module):
    def __init__(self, alpha=0.7, delta=0.05, window_size=5, temporal_weight=0.1):
        super().__init__()
        # ------------------------------------------------------------------------------
        # ALPHA (Controls balance between Huber Loss and SSIM)
        # More alpha  → Model focuses on voxel accuracy (Huber loss)
        # Less alpha  → Model prioritizes structural similarity (SSIM)
        # ------------------------------------------------------------------------------
        # Recommended tuning:
        # - 0.6 - 0.9 → Noisy data (CT/MRI with artifacts) → More Huber
        # - 0.4 - 0.7 → Sharp structures (CT/MRI edges)  → Balance of both
        # - 0.3 - 0.5 → Blurry predictions → More SSIM for finer details
        # Default: alpha = 0.7 (Strong Huber, some SSIM)
        # ------------------------------------------------------------------------------
        self.alpha = alpha  # More alpha = More reliance on Huber
        # ------------------------------------------------------------------------------
        # DELTA (Threshold where Huber Loss switches from MSE-like to MAE-like behavior)
        # - Higher delta → More sensitive to small errors (acts like MSE)
        # - Lower delta  → More resistant to outliers (acts like MAE)
        # ------------------------------------------------------------------------------
        # Recommended tuning:
        # - > 0.05  → High-noise data (CT/MRI with artifacts) → More robust to outliers
        # - < 0.05  → Low-noise data (Well-normalized, synthetic) → More sensitive to details
        # - 0.02 - 0.05  → If predictions are too blurry
        # Default: delta = 0.05 (or dynamically adjusted per epoch)
        # ------------------------------------------------------------------------------
        self.delta = delta  # This will be dynamically updated every epoch
        # ------------------------------------------------------------------------------
        # TEMPORAL WEIGHT (Penalizes abrupt voxel intensity changes between frames)
        # - Higher value → Forces smoother transitions
        # - Lower value  → Allows more flexibility in voxel changes
        # ------------------------------------------------------------------------------
        # Recommended tuning:
        # - > 0.2  → Strong penalty for sudden intensity jumps (Flickering reduction)
        # - 0.05 - 0.1  → Best for smoothly changing sequences (MRI, Weather, Fluids)
        # - < 0.05  → Allows more dynamic changes (if model is too rigid)
        # Default: temporal_weight = 0.1 (balanced smoothness)
        # ------------------------------------------------------------------------------
        self.temporal_weight = temporal_weight  # More weight = Stronger smoothness enforcement

        #window_size means padding
        self.ssim_module = kornia.losses.SSIMLoss(window_size=window_size, reduction="mean")

    def temporal_smoothness_loss(self, y_pred):
        #Penalizes sudden changes over time by computing L1 loss between consecutive frames.
        #So the object should remain stationary over time
        return torch.mean(torch.abs(y_pred[:, 1:] - y_pred[:, :-1]))  # Difference between t and t+1

    def forward(self, y_pred, y_true):
        B, T, C, H, W = y_pred.shape

        # Swap time and channel dimensions: [B, C*T, H, W]
        y_pred_restructured = y_pred.permute(0, 2, 1, 3, 4).reshape(B, C * T, H, W)
        y_true_restructured = y_true.permute(0, 2, 1, 3, 4).reshape(B, C * T, H, W)

        # Computes the SSIM treating time as another spatial dimension
        ssim_loss = self.ssim_module(y_pred_restructured, y_true_restructured)

        # Computes the Huber loss with the adjusted delta
        huber_loss = F.huber_loss(y_pred, y_true, delta=self.delta, reduction="mean")

        # Compute the Temporal Smoothness Loss
        temporal_loss = self.temporal_smoothness_loss(y_pred)

        # Weighted combination
        total_loss = self.alpha * huber_loss + (1 - self.alpha) * ssim_loss + self.temporal_weight * temporal_loss
        return total_loss, huber_loss, ssim_loss, temporal_loss

class HuberSSIMLoss3D(nn.Module):
    def __init__(self, alpha=0.7, delta=0.05, window_size=5, temporal_weight=0.1):
        super().__init__()
        # ------------------------------------------------------------------------------
        # ALPHA (Controls balance between Huber Loss and SSIM)
        # More alpha  → Model focuses on voxel accuracy (Huber loss)
        # Less alpha  → Model prioritizes structural similarity (SSIM)
        # ------------------------------------------------------------------------------
        # Recommended tuning:
        # - 0.6 - 0.9 → Noisy data (CT/MRI with artifacts) → More Huber
        # - 0.4 - 0.7 → Sharp structures (CT/MRI edges)  → Balance of both
        # - 0.3 - 0.5 → Blurry predictions → More SSIM for finer details
        # Default: alpha = 0.7 (Strong Huber, some SSIM)
        # ------------------------------------------------------------------------------
        self.alpha = alpha  # More alpha = More reliance on Huber
        # ------------------------------------------------------------------------------
        # DELTA (Threshold where Huber Loss switches from MSE-like to MAE-like behavior)
        # - Higher delta → More sensitive to small errors (acts like MSE)
        # - Lower delta  → More resistant to outliers (acts like MAE)
        # ------------------------------------------------------------------------------
        # Recommended tuning:
        # - > 0.05  → High-noise data (CT/MRI with artifacts) → More robust to outliers
        # - < 0.05  → Low-noise data (Well-normalized, synthetic) → More sensitive to details
        # - 0.02 - 0.05  → If predictions are too blurry
        # Default: delta = 0.05 (or dynamically adjusted per epoch)
        # ------------------------------------------------------------------------------
        self.delta = delta  # This will be dynamically updated every epoch
        # ------------------------------------------------------------------------------
        # TEMPORAL WEIGHT (Penalizes abrupt voxel intensity changes between frames)
        # - Higher value → Forces smoother transitions
        # - Lower value  → Allows more flexibility in voxel changes
        # ------------------------------------------------------------------------------
        # Recommended tuning:
        # - > 0.2  → Strong penalty for sudden intensity jumps (Flickering reduction)
        # - 0.05 - 0.1  → Best for smoothly changing sequences (MRI, Weather, Fluids)
        # - < 0.05  → Allows more dynamic changes (if model is too rigid)
        # Default: temporal_weight = 0.1 (balanced smoothness)
        # ------------------------------------------------------------------------------
        self.temporal_weight = temporal_weight  # More weight = Stronger smoothness enforcement

        #window_size means padding
        self.ssim_module = kornia.losses.SSIM3DLoss(window_size=window_size, reduction="mean")

    def temporal_smoothness_loss(self, y_pred):
        #Penalizes sudden changes over time by computing L1 loss between consecutive frames.
        #So the object should remain stationary over time
        return torch.mean(torch.abs(y_pred[:, 1:] - y_pred[:, :-1]))  # Difference between t and t+1

    def forward(self, y_pred, y_true):
        B, T, C, D, H, W = y_pred.shape

        # Swap time and channel dimensions: [B, C*T, D, H, W]
        y_pred_restructured = y_pred.permute(0, 2, 1, 3, 4, 5).reshape(B, C * T, D, H, W)
        y_true_restructured = y_true.permute(0, 2, 1, 3, 4, 5).reshape(B, C * T, D, H, W)

        # Computes the SSIM treating time as another spatial dimension
        ssim_loss = self.ssim_module(y_pred_restructured, y_true_restructured)

        # Computes the Huber loss with the adjusted delta
        huber_loss = F.huber_loss(y_pred, y_true, delta=self.delta, reduction="mean")

        # Compute the Temporal Smoothness Loss
        temporal_loss = self.temporal_smoothness_loss(y_pred)

        # Weighted combination
        total_loss = self.alpha * huber_loss + (1 - self.alpha) * ssim_loss + self.temporal_weight * temporal_loss
        return total_loss, huber_loss, ssim_loss, temporal_loss

In [8]:
class Pl_Model(pl.LightningModule):
    def __init__(
        self,
        passed_model: nn.Module,
        config: Dict[str, Any],
    ):
        super(Pl_Model, self).__init__()
        self.passed_model = passed_model
        self.config = config
        # ------------------------------------------------------------------------------
        # DELTA FACTOR (Scales how much delta is updated per epoch)
        # - Higher delta_factor → More aggressive updates to delta
        # - Lower delta_factor  → Smoother, slower changes to delta
        # ------------------------------------------------------------------------------
        # Recommended tuning:
        # - > 1.5  → If delta is too unstable (jumps too much)
        # - 1.2 - 1.5  → Best for gradual adaptation (Default)
        # - < 1.2  → If delta changes too slowly (use for very stable datasets)
        # ------------------------------------------------------------------------------
        # Default: delta_factor = 1.2 (Balanced adaptation)
        # ------------------------------------------------------------------------------
        self.delta_factor = 1.2  # More factor = Faster delta adjustments
        self.delta = 0.05
        self.previous_delta = self.delta
        
        #speicher alle parameter ab
        self.save_hyperparameters(ignore=["passed_model"])

        # Setup training components
        self.mse_criterion = nn.MSELoss()
        self.psnr_criterion = PeakSignalNoiseRatio()
        self.huberssim2d_criterion = HuberSSIMLoss2D()
        self.huberssim3d_criterion = HuberSSIMLoss3D()
        self.huber_criterion = nn.HuberLoss()

    def forward(self, x):
        x = self.passed_model(x)
        #Tanh has a larger gradient range, reducing saturation issues compared to sigmoid.
        #Allows more stable gradient flow for deep networks.
        x = 0.5*(F.tanh(x)+1)
        return x

    def configure_optimizers(self):
        """Sets the Optimizer for the Model"""
        optimizer = optim.AdamW(
            self.parameters(), 
            lr=config['learning_rate'],
        )
        return [optimizer]

    def _calculate_loss(self, batch, mode="train"):
        """Calculates the loss for a batch in different modes (training, validation, testing)"""
        inputs, targets = batch

        #forward pass
        mse_loss = 0.0
        huber_loss = 0.0
        rmse_loss = 0.0
        ssim_loss = 0.0
        huberssim_loss = 0.0
        temporal_loss = 0.0
        psnr_loss = 0.0
        total_loss = 0.0

        #mse_loss_table = []
        for t in range(0, self.config["pred_frames"], self.config["pred_n_frames_per_step"]):
            if self.config["pred_frames"]-t<self.config["pred_n_frames_per_step"]:
                frames_this_step = self.config["pred_frames"]-t
            else:
                frames_this_step = self.config["pred_n_frames_per_step"]
            outputs = self.forward(inputs)
            #print(f"{t}:{t+frames_this_step}")
            #get only the first predicted frame
            outputs = outputs[:, :frames_this_step, :, :]
            
            #calcualte losses
            mse_loss_ = self.mse_criterion(outputs, targets[:, t:t+frames_this_step, :, :])
            rmse_loss_ = torch.sqrt(self.mse_criterion(outputs, targets[:, t:t+frames_this_step, :, :]))
            psnr_loss_ = self.psnr_criterion(outputs, targets[:, t:t+frames_this_step, :, :])
            huberssim_loss_, huber_loss_, ssim_loss_, temporal_loss_ = self.huberssim2d_criterion(outputs, targets[:, t:t+frames_this_step, :, :])
            total_loss_ = huberssim_loss_ #self.huber_criterion(outputs, targets[:, t:t+frames_this_step, :, :])  

            #mse_loss_table.append([t, mse_loss_.item()])
            
            mse_loss += mse_loss_
            huber_loss += huber_loss_
            rmse_loss += rmse_loss_
            ssim_loss += ssim_loss_
            huberssim_loss += huberssim_loss_
            temporal_loss += temporal_loss_
            psnr_loss += psnr_loss_
            total_loss += total_loss_
            
            inputs = torch.cat([inputs[:, self.config["pred_n_frames_per_step"]:, :, :], outputs], dim=1)

        #logging
        self.log(f"{mode}_mse_loss", mse_loss)
        self.log(f"{mode}_huber_loss", huber_loss)
        self.log(f"{mode}_rmse_loss", rmse_loss)
        self.log(f"{mode}_ssim_loss", ssim_loss)
        self.log(f"{mode}_huberssim_loss", huberssim_loss)
        self.log(f"{mode}_temporal_loss", temporal_loss)
        self.log(f"{mode}_psnr_loss", psnr_loss)
        self.log(f"{mode}_total_loss", total_loss, prog_bar=True)

        #table logging
        #wandb.log({"Loss per Timestep": wandb.plot.line(wandb.Table(data=mse_loss_table, columns=["Timestep", "Loss"]), "Timestep", "Loss", title="Loss per Timestep")})

        return total_loss

    def training_step(self, batch, batch_idx):
        loss = self._calculate_loss(batch, mode="train")
        return loss

    @torch.no_grad()
    def validation_step(self, batch, batch_idx, dataloader_idx):
        if dataloader_idx==1:
            inputs, targets = batch
            # iterate over depth
            outputs = []
            for d in range(inputs.shape[3]):
                inputs_ = inputs[:, :, :, d, :, :]
                #iterate over time
                outputs_d = []
                for t in range(0, inputs.shape[1], self.config["pred_n_frames_per_step"]):
                    if inputs.shape[1]-t<self.config["pred_n_frames_per_step"]:
                        frames_this_step = inputs.shape[1]-t
                    else:
                        frames_this_step = self.config["pred_n_frames_per_step"]
                    
                    outputs_t = self.forward(inputs_)
            
                    #get only the first predicted frame
                    outputs_t = outputs_t[:, :frames_this_step, :, :]
            
                    #add to depth lst
                    outputs_d.append(outputs_t)
            
                    inputs_ = torch.cat([inputs_[:, self.config["pred_n_frames_per_step"]:, :, :], outputs_t], dim=1)
            
                #concat time and add to overall lst
                outputs_d = torch.concat(outputs_d, dim=1)
                outputs.append(outputs_d)
            #stack over depth
            outputs = torch.stack(outputs, dim=3)

            #calculate losses
            mse_loss = self.mse_criterion(outputs, targets)
            rmse_loss = torch.sqrt(self.mse_criterion(outputs, targets))
            huberssim_loss, huber_loss, ssim_loss, temporal_loss = self.huberssim3d_criterion(outputs, targets)
            psnr_loss = self.psnr_criterion(outputs, targets)
            total_loss = huberssim_loss#self.huber_criterion(outputs, targets)

            #logging
            self.log(f"overall_val_mse_loss", mse_loss)
            self.log(f"overall_val_huber_loss", huber_loss)
            self.log(f"overall_val_rmse_loss", rmse_loss)
            self.log(f"overall_val_ssim_loss", ssim_loss)
            self.log(f"overall_val_huberssim_loss", huberssim_loss)
            self.log(f"overall_val_temporal_loss", temporal_loss)
            self.log(f"overall_val_psnr_loss", psnr_loss)
            self.log(f"overall_val_total_loss", total_loss)
                    
        else:
            _ = self._calculate_loss(batch, mode="val")

    @torch.no_grad()
    def test_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="test")
    
    def on_train_epoch_end(self):
        #adjust delta
        val_loader = self.trainer.datamodule.val_dataloader()[0]
        all_errors = []
    
        with torch.no_grad():  
            for batch in val_loader:
                x, y = batch
                y_pred = self(x.to(self.device))
                error = torch.abs(y.to(self.device) - y_pred)
                all_errors.append(error.view(-1))
    
        all_errors = torch.cat(all_errors)
        new_delta = self.delta_factor * torch.std(all_errors).item()

        #Blend previous and new delta for smoother updates
        #is capped between 0.02 and 0.35 so that is the data is too noisy huber does not just become mse
        new_delta = min(0.5, max(0.02, 0.8 * self.previous_delta + 0.2 * new_delta))
        self.previous_delta = new_delta

        #update
        self.huberssim2d_criterion.delta = new_delta
        self.huberssim3d_criterion.delta = new_delta
    
        #logging
        self.log("delta", new_delta)
        
    
    @torch.no_grad()
    def check_losses(self, loader, mode, use_wandb=False):
        mse_loss = 0.0
        huber_loss = 0.0
        rmse_loss = 0.0
        ssim_loss = 0.0
        huberssim_loss = 0.0
        temporal_loss = 0.0
        psnr_loss = 0.0
        total_loss = 0.0
        for inputs, targets in loader:
            for t in range(self.config["pred_frames"]):
                mse_loss_ = self.mse_criterion(inputs[:, -1, :, :, :].unsqueeze(1), targets[:, t, :, :].unsqueeze(1))
                huberssim_loss_, huber_loss_, ssim_loss_, temporal_loss_ = self.huberssim2d_criterion(inputs[:, -1, :, :, :].unsqueeze(1), targets[:, t, :, :].unsqueeze(1))
                rmse_loss_ = torch.sqrt(self.mse_criterion(inputs[:, -1, :, :, :].unsqueeze(1), targets[:, t, :, :].unsqueeze(1)))
                psnr_loss_ = self.psnr_criterion(inputs[:, -1, :, :, :].unsqueeze(1), targets[:, t, :, :].unsqueeze(1))
                total_loss_ = huberssim_loss_   
                
                mse_loss += mse_loss_.item()
                huber_loss += huber_loss_.item()
                rmse_loss += rmse_loss_.item()
                ssim_loss += ssim_loss_.item()
                huberssim_loss += huberssim_loss_.item()
                temporal_loss += temporal_loss_.item()
                psnr_loss += psnr_loss_.item()
                total_loss += total_loss_.item()
                
        mse_loss = mse_loss / len(loader)
        huber_loss = huber_loss / len(loader)
        rmse_loss = rmse_loss / len(loader)
        ssim_loss = ssim_loss / len(loader)
        huberssim_loss = huberssim_loss / len(loader)
        temporal_loss = temporal_loss / len(loader)
        psnr_loss = psnr_loss / len(loader)
        total_loss = total_loss / len(loader)

        if use_wandb:
            wandb.log({f"Checked_{mode}_mse_loss": mse_loss})
            wandb.log({f"Checked_{mode}_huber_loss": huber_loss})
            wandb.log({f"Checked_{mode}_rmse_loss": rmse_loss})
            wandb.log({f"Checked_{mode}_ssim_loss": ssim_loss})
            wandb.log({f"Checked_{mode}_huberssim_loss": huberssim_loss})
            wandb.log({f"Checked_{mode}_temporal_loss": temporal_loss})
            wandb.log({f"Checked_{mode}_psnr_loss": psnr_loss})
            wandb.log({f"Checked_{mode}_total_loss": total_loss})
        
        return mse_loss, huber_loss, ssim_loss, huberssim_loss, temporal_loss, rmse_loss, psnr_loss, total_loss
        
    def log_predictions(self):
        """Log example predictions to wandb"""
        #needs to be added to other method
        if epoch % self.config['viz_interval'] == 0:
                self.log_predictions()
        #but this whole method needs to be rewritten
        self.model.eval()
        with torch.no_grad():
            # Get a batch of validation data
            data, target = next(iter(self.val_loader))
            data = data.to(self.device)
            target = target.to(self.device)
            
            # Generate predictions
            output = self.model(data)
            
            # Log images
            wandb.log({
                "predictions": wandb.Image(output[0, 0].cpu()),
                "targets": wandb.Image(target[0, 0].cpu()),
                "input_sequence": [wandb.Image(data[0, i].cpu()) for i in range(data.shape[1])]
            })

#### Config

In [9]:
config = {
    "root": "../NormalizedQualityFiltered",
    'batch_size': 29,
    'learning_rate': 0.0001,
    "num_workers": 10,#0, wenn die gpu nicht benutzt wird
    "pin_memory": True if torch.cuda.is_available() else False,#False, wenn die gpu nicht benutzt wird
    "drop_last": False,
    'epochs': 40,
    #'log_interval': 20,
    #'viz_interval': 1,
    'run_name': '2D-SimVP',
    'input_frames': 9,
    "pred_frames": 9,
    "pred_n_frames_per_step": 9,
    'base_filters': 32,
    "train_split": 0.7,
    "val_split": 0.15,
    "test_split": 0.15,
}
config["run_name"] += f"_{config['pred_frames']}"
if config["pred_frames"] == config["pred_n_frames_per_step"]:
    config["run_name"] += "_NAR"
elif config["pred_n_frames_per_step"] == 1:
    config["run_name"] += "_FAR"
else:
    config["run_name"] += f"_PAR_{config['pred_n_frames_per_step']}"

# Get data loaders
'''train_loader, val_loader, test_loader, val_loader_3d, test_loader_3d, train_dataset, val_dataset, test_dataset, val_dataset_3d, test_dataset_3d = get_data_loaders(
    batch_size=config['batch_size'],
    num_workers=config["num_workers"],
    pin_memory=config["pin_memory"],
    drop_last=config["drop_last"],
    sequence_length=config["input_frames"], 
    prediction_length=config["pred_frames"],
)'''
dm = VolumeDataModule(
    root=config["root"],
    batch_size=config['batch_size'],
    num_workers=config["num_workers"],
    pin_memory=config["pin_memory"],
    drop_last=config["drop_last"],
    sequence_length=config["input_frames"],
    prediction_length=config["pred_frames"],
    train_split=config["train_split"],
    val_split=config["val_split"],
    test_split=config["test_split"],
)

wandb_logger = WandbLogger(entity="ChadCTP", project="perfusion-ct-prediction", name=config["run_name"])

# Initialize model for tuning
model = SimVP(
    shape_in=[config["input_frames"], 1, 256, 256],
    # hid_S=128, 
    # hid_T=256, 
    # N_S=8, 
    # N_T=8, 
    # incep_ker=[3,5,7,11], 
    # groups=8,
)

# Initialize pl_model for tuning
pl_model = Pl_Model(
    passed_model=model,
    config=config,
)

checkpoint_callback = ModelCheckpoint(
    monitor="val_total_loss/dataloader_idx_0",  
    mode="min",  
    save_top_k=1,  
    filename="best-checkpoint",
    verbose=True,
)

# Initialize trainer for tuning
trainer = pl.Trainer(
    logger=wandb_logger,
    accelerator="gpu",
    devices= [1] if torch.cuda.is_available() else None,
    max_epochs=config["epochs"],
    callbacks=[RichProgressBar(), checkpoint_callback],
    check_val_every_n_epoch=5,
)

#wandb_logger.watch(pl_model, log_graph=False)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(
    model=pl_model,
    datamodule=dm,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name                  ┃ Type                 ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ passed_model          │ SimVP                │ 13.8 M │ train │
│ 1 │ mse_criterion         │ MSELoss              │      0 │ train │
│ 2 │ psnr_criterion        │ PeakSignalNoiseRatio │      0 │ train │
│ 3 │ huberssim2d_criterion │ HuberSSIMLoss2D      │      0 │ train │
│ 4 │ huberssim3d_criterion │ HuberSSIMLoss3D      │      0 │ train │
│ 5 │ huber_criterion       │ HuberLoss            │      0 │ train │
└───┴───────────────────────┴──────────────────────┴────────┴───────┘

Trainable params: 13.8 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 13.8 M                                                                                               
Total estimated model params size (MB): 55                                                                         
Modules in train mode: 360                                                                                         
Modules in eval mode: 0

Output()

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches 
(46) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if
you want to see logs for the training epoch.

Epoch 4, global step 230: 'val_total_loss/dataloader_idx_0' reached 0.09117 (best 0.09117), saving model to './perfusion-ct-prediction/oqelujc5/checkpoints/best-checkpoint.ckpt' as top 1
Epoch 9, global step 460: 'val_total_loss/dataloader_idx_0' reached 0.05997 (best 0.05997), saving model to './perfusion-ct-prediction/oqelujc5/checkpoints/best-checkpoint.ckpt' as top 1
Epoch 14, global step 690: 'val_total_loss/dataloader_idx_0' reached 0.02429 (best 0.02429), saving model to './perfusion-ct-prediction/oqelujc5/checkpoints/best-checkpoint.ckpt' as top 1
Epoch 19, global step 920: 'val_total_loss/dataloader_idx_0' reached 0.01268 (best 0.01268), saving model to './perfusion-ct-prediction/oqelujc5/checkpoints/best-checkpoint.ckpt' as top 1
Epoch 24, global step 1150: 'val_total_loss/dataloader_idx_0' reached 0.00959 (best 0.00959), saving model to './perfusion-ct-prediction/oqelujc5/checkpoints/best-checkpoint.ckpt' as top 1
Epoch 29, global step 1380: 'val_total_loss/dataloader_idx_0'

In [11]:
#check and log the losses "to beat"
dm.setup()
pl_model.check_losses(dm.train_dataloader(), mode="train", use_wandb=True)
pl_model.check_losses(dm.val_dataloader()[0], mode="val", use_wandb=True)
pl_model.check_losses(dm.test_dataloader(), mode="test", use_wandb=True)

#val_results = trainer.validate(pl_model, datamodule=dm)
test_results = trainer.test(pl_model, datamodule=dm)

save_load_path = f"../ModelWeights/{config['run_name']}.ckpt"
trainer.save_checkpoint(save_load_path)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_huber_loss      │  4.0836679545463994e-05   │
│    test_huberssim_loss    │   0.006307575851678848    │
│       test_mse_loss       │   0.0002555459795985371   │
│      test_psnr_loss       │     36.12557601928711     │
│      test_rmse_loss       │    0.01579868607223034    │
│      test_ssim_loss       │   0.020451875403523445    │
│    test_temporal_loss     │   0.0014342714566737413   │
│      test_total_loss      │   0.006307575851678848    │
└───────────────────────────┴───────────────────────────┘

In [12]:
#testing
pl_model = Pl_Model.load_from_checkpoint(
    "./perfusion-ct-prediction/oqelujc5/checkpoints/best-checkpoint.ckpt",
    passed_model=model,
)

dm.setup()
test_dataloader = dm.test_dataloader()

print(len(test_dataloader))
pl_model.to(device)
"""for inputs, targets in test_dataloader:
    outputs = pl_model.forward(inputs.to(device))
    outputs = outputs.detach().cpu()
    outputs = torch.concat([inputs, outputs], dim=1).squeeze(0, 2).numpy()
    targets = torch.concat([inputs, targets], dim=1).squeeze(0, 2).numpy()
    print(outputs.shape)
    print(targets.shape)
    break"""
# method 1
#inputs, targets = next(iter(test_dataloader))

# method 2
# only for single shot
vol = torch.tensor(np.load("../NormalizedQualityFiltered/MOL-061.npy")).unsqueeze(1)
inputs = vol[0:9]#.unsqueeze(0)
targets = vol[9:]#.unsqueeze(0)
print(inputs.shape)
inputs_ = torch.concat([inputs.transpose(0, 2).transpose(1, 2), torch.zeros([config["batch_size"]-vol.shape[2], config["input_frames"], 1, vol.shape[-2], vol.shape[-1]])])
outputs = pl_model(inputs_.to(device))
outputs = outputs[:vol.shape[2]]
print(outputs.shape)
outputs = outputs.detach().cpu()
outputs = torch.concat([inputs, outputs.transpose(0, 2).transpose(0, 1)], dim=0).squeeze(1).numpy()
targets = torch.concat([inputs, targets], dim=1).squeeze(1).numpy()

print(outputs.shape)
print(targets.shape)
#for i in range(inputs.shape[2]):
#    print(inputs[:, :, i, :, :].shape)

np.save(f"outputs_{config["run_name"]}.npy", outputs)
np.save(f"targets_{config["run_name"]}.npy", targets)

RuntimeError: Error(s) in loading state_dict for Pl_Model:
	Unexpected key(s) in state_dict: "passed_model.enc.enc.4.conv.conv.weight", "passed_model.enc.enc.4.conv.conv.bias", "passed_model.enc.enc.4.conv.norm.weight", "passed_model.enc.enc.4.conv.norm.bias", "passed_model.enc.enc.5.conv.conv.weight", "passed_model.enc.enc.5.conv.conv.bias", "passed_model.enc.enc.5.conv.norm.weight", "passed_model.enc.enc.5.conv.norm.bias", "passed_model.enc.enc.6.conv.conv.weight", "passed_model.enc.enc.6.conv.conv.bias", "passed_model.enc.enc.6.conv.norm.weight", "passed_model.enc.enc.6.conv.norm.bias", "passed_model.enc.enc.7.conv.conv.weight", "passed_model.enc.enc.7.conv.conv.bias", "passed_model.enc.enc.7.conv.norm.weight", "passed_model.enc.enc.7.conv.norm.bias", "passed_model.dec.dec.4.conv.conv.weight", "passed_model.dec.dec.4.conv.conv.bias", "passed_model.dec.dec.4.conv.norm.weight", "passed_model.dec.dec.4.conv.norm.bias", "passed_model.dec.dec.5.conv.conv.weight", "passed_model.dec.dec.5.

In [ ]:
@torch.no_grad()
def overall_loss(model, loader, device):
    mse_loss = 0.0
    huber_loss = 0.0
    rmse_loss = 0.0
    #ssim_loss = 0.0
    psnr_loss = 0.0
    total_loss = 0.0
    model = model.to(device)
    for inputs, targets in loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        # iterate over depth
        outputs = []
        for d in range(inputs.shape[3]):
            inputs_ = inputs[:, :, :, d, :, :]
            #iterate over time
            outputs_d = []
            for t in range(0, model.config["pred_frames"], model.config["pred_n_frames_per_step"]):
                if model.config["pred_frames"]-t<model.config["pred_n_frames_per_step"]:
                    frames_this_step = model.config["pred_frames"]-t
                else:
                    frames_this_step = model.config["pred_n_frames_per_step"]
                
                outputs_t = model.forward(inputs_)
        
                #get only the first predicted frame
                outputs_t = outputs_t[:, :frames_this_step, :, :]

                #add to depth lst
                outputs_d.append(outputs_t)
        
                inputs_ = torch.cat([inputs_[:, model.config["pred_n_frames_per_step"]:, :, :], outputs_t], dim=1)
            #concat time and add to overall lst
            outputs_d = torch.concat(outputs_d, dim=1)
            outputs.append(outputs_d)
    
        #stack over depth
        outputs = torch.stack(outputs, dim=3)
        #print(outputs.shape)
        
        #calculate losses
        mse_loss += model.mse_criterion(outputs, targets).item()
        huber_loss += model.huber_criterion(outputs, targets).item()
        rmse_loss += torch.sqrt(model.mse_criterion(outputs, targets)).item()
        #ssim_loss = model.ssim_criterion(outputs, targets).item()
        psnr_loss += model.psnr_criterion(outputs, targets).item()
        total_loss += huber_loss

    mse_loss = mse_loss / len(loader)
    huber_loss = huber_loss / len(loader)
    rmse_loss = rmse_loss / len(loader)
    #ssim_loss = ssim_loss / len(loader)
    psnr_loss = psnr_loss / len(loader)
    total_loss = total_loss / len(loader)

    return outputs, mse_loss, huber_loss, rmse_loss, psnr_loss, total_loss

dm.setup()
_, mse_loss, huber_loss, rmse_loss, psnr_loss, total_loss = overall_loss(model=pl_model, loader=dm.test_dataloader_3d(), device=device)
mse_loss, huber_loss, rmse_loss, psnr_loss, total_loss

In [ ]:
'''# idk tuning behaves really weird and different from fit regarding memory usage
#tuning
tuner = Tuner(trainer)
tuner.scale_batch_size(pl_model, datamodule=dm, mode="binsearch")

#cleaning up
del tuner
del trainer
del pl_model
del model
gc.collect()
torch.cuda.empty_cache()
if torch.cuda.is_available():
    torch.cuda.ipc_collect()

# Reinitialize model for tuning
model = SimVP(
    shape_in=[config["input_frames"], 1, 256, 256]
)

# Reinitialize pl_model for training
pl_model = Pl_Model(
    passed_model=model,
    config=config,
)

# Reinitialize trainer for training
trainer = pl.Trainer(
    logger=wandb_logger,
    accelerator="gpu",
    devices= [2] if torch.cuda.is_available() else None,
    max_epochs=config["epochs"],
    callbacks=[RichProgressBar()],
    check_val_every_n_epoch=1,
    #enable_checkpointing=False,
)'''